# Introduction
Infrastructures nowadays are generating data at such a rapid rate and great volume that it is sometimes impossible to handle them in traditional ways. Fortunately, programming tools such Python include many powerful libraries that can assist the data wrangling process.

In this exercise, you will learn to read and analyze tabular and tree-structured data. Both of which are among the most popular types of inputs you will get for your work or research. Specifically, we will show how to handle `.csv`-like files using the Pandas library (https://pandas.pydata.org/) and perform tasks such as imputing missing data, looping through each row and merging two tables. The Pandas library also works well with other tabular data sources, such as the popular `.xlsx` file. On the other hand, Python can also work with semi-structured tree-like data (e.g., `.json` or `.xml`) using standard libraries (https://docs.python.org/3/library/). Pandas is not a Python standard library, thus it usually requires installation. The `json` and `xml` libaries are included in the Python standard libraries and do not require additional installation.

Even though tools such as Pandas is handy and efficient, it should be noted that data processing (cleaning, merging, validation, etc.), or sometimes even making sense of the data, can still take the majority of the time in an infrastructure analysis project.

# Tabular data (.csv, .xlsx)
You are probably familiar with tabular data by using tools such as MS Excel or Google Spreadsheets. If we view each row as a separate record (e.g., the three rows in the table below represent three separate roads), then all records share the same set of attributes, (e.g., all roads have five attributes: ID, name, direction, start_intersection, end_intersection). Because of this highly structured format, there is no need to store the name of attributes for each record, which saves a lot of space for large data files compared to `.json` or `.xml` file that will be introduced later.


| ID | name          | direction      | start_intersection | end_intersection |
|----|---------------|----------------|--------------------|------------------|
| 1  | Hearst Avenue | East to West   | La Loma Avenue     | Oxford Street    |
| 2  | Hearst Avenue | West to East   | Oxford Street      | La Loma Avenue   |
| 3  | Euclid Avenue | South to North | Hearst Avenue      | Marin Avenue     |

* Note: the two directions of the same road between two common intersections are usually regarded as separate road links in traffic analysis.

## Reading data
In the folder of today's exercise, we provided an example `hearst_euclid.csv` file, which contains some information of the two streets on the Northside of the Berkeley campus. Feel free to download it and open in text editor or Excel for this exercise. Now, let's read the same file into a Pandas dataframe.

In [1]:
# import the pandas library
import pandas as pd 

# read the csv file by file name
df = pd.read_csv('hearst_euclid.csv')

# print the first few lines of the data
df.head()

,start_igraph,end_igraph,road_name,direction
0,1633,1631,Hearst_Ave,Eastward
1,1631,4552,Hearst_Ave,Eastward
2,4552,1240,Hearst_Ave,Eastward
3,1240,12798,Hearst_Ave,Eastward
4,12798,1630,Hearst_Ave,Eastward


## Exploration: shape and unique
Now let's do some basic exploration:
- How many rows/columns are there in the data?
- How many different roads are in the data?

Previously, we have read the csv file into the `df` variable, where `df` is short for `dataframe` (think dataframe like an excel table, or the two-dimensional array data frame in software `R`). You can name the variable whatever you like, but `df` is typical for variables that are pandas dataframes.

Each pandas dataframe object has the `shape` property, which can be accessed using syntax such as `df.shape`. The first element of the `shape` property is the number of rows in the dataframe, and the second element is the number of columns.

To focus on a particular column in a dataframe (which is called a `series`), you can access it using `df[column_name]`. Series has a method called `unique()` that returns the unique values in that column. Notice the `()` after the method, which is not there if we want to access properties such as `shape`.

As you will soon find out, our data has two roads and each road has two directions. However, for each road-direction, the road is further broken down into smaller segments, thus the total number of records are larger than four.

In [2]:
# print the numbers of rows and columns in the data
print( '# rows and columns', df.shape )

# print the names of unique roads in the data
print( 'Unique roads: ', df['road_name'].unique() )

# rows and columns (79, 4)
Unique roads:  ['Hearst_Ave' 'Euclid_Ave']


## Selecting data
Sometimes, you may want to choose a subset of the data to work with. In this case, you will need to select the data based on some criteria (e.g, give me the `n`-th row or give me all records whose `name` is `Hearst_Ave`). Index in Pandas (as well as in Python) is 0-based, meaning the row id of the first record is 0, the row id of the second record is 1, and so forth. Selecting data in pandas is usually performed by `df.loc[]` (label based) or `df.iloc[]` (integer position based). Or use `df[column_name]` is also handy to retrieve a certain column in the dataframe. We will explain how to use them in the exercise below.

If you want to index the data in other ways than introduced below, this page provides further information https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [3]:
# selecting a single row by row id: the 1st row
display( df.iloc[0:1] ) ### based on index position

# selecting a single row by row id: the 10th row
display( df.iloc[9:10] )

# selecting a single cell row and column id: the 1st column in the first row
display( df.iloc[0,0] )

# selecting a single cell based on row id and column name
display( df['road_name'].iloc[41] )

# selecting multiple rows by condition
# since the subset of the data has many rows, let's just print the first two rows
display( df.loc[ df['road_name']=='Euclid_Ave' ].head(2) )

# selecting multiple columns by column name: use a list to specify all columns of interest
display( df[ ['start_igraph', 'road_name', 'direction'] ].head(2) )

,start_igraph,end_igraph,road_name,direction
0,1633,1631,Hearst_Ave,Eastward


,start_igraph,end_igraph,road_name,direction
9,1626,1623,Hearst_Ave,Eastward


1633

'Euclid_Ave'

,start_igraph,end_igraph,road_name,direction
41,2163,13700,Euclid_Ave,Southward
42,13700,5827,Euclid_Ave,Southward


,start_igraph,road_name,direction
0,1633,Hearst_Ave,Eastward
1,1631,Hearst_Ave,Eastward


## Looping through the data
In the example data, there are two road names (`Hearst_Ave` and `Euclid_Ave`, while each road has two directions and multiple segments). We now want to create a new column, `road_and_direction`, which combines the first letter in the `road_name` column and `direction` column, e.g., `H_W` for the westward direction of Hearst avenue, or `E_S` for the southward direction of Euclid Avenue.

There are different methods to apply the same function to each record in a dataframe. However, for beginners, it is easier to just think of this task as looping through each row of the dataframe (using `iterrows()` or `itertuples()`) and performing the same operations on each row. The performance is usually not worse than more sophisticated methods (unless you are doing simple algebra calculations, which can be vectorized and run much faster with NumPy, see https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6).

We will introduce the `itertuples()` method here. It is typically used with a `for` loop. Inside the `for` loop, you can get the value under a specific column of the current row with `getattr()` method.

In [4]:
# first, create a new, empty list called `road_and_direction`
# this is because usually we don't want to update the dataframe directy in a loop
road_and_direction = []

for row in df.itertuples():
    
    # get the first letter (remember, the index begins at 0 in Python) of the `road_name` column
    r = getattr(row, 'road_name')[0]
    d = getattr(row, 'direction')[0]
    r_d = r + '_' + d ### you can concatenate strings with the `+` sign
    road_and_direction.append( r_d )

# create the road_and_direction column
df['road_and_direction'] = road_and_direction

# check results at random locations
display(df.iloc[[0, 1, 50, -2, -1]])

,start_igraph,end_igraph,road_name,direction,road_and_direction
0,1633,1631,Hearst_Ave,Eastward,H_E
1,1631,4552,Hearst_Ave,Eastward,H_E
50,1151,3441,Euclid_Ave,Southward,E_S
77,5827,13700,Euclid_Ave,Northward,E_N
78,13700,2163,Euclid_Ave,Northward,E_N


## Merging
You may want to combine two tables into a same dataframe. For example, our table looks like

|     | start_igraph  | end_igraph     | road_name   | direction   | road_and_direction |
|-----|---------------|----------------|-------------|-------------|--------------------|
| 1   | 1633          | 1631           | Hearst_Ave  | Eastward    | H_E                |
| 2   | 1631          | 4552           | Hearst_Ave  | Eastward    | H_E                |
| ..  | ..            | ..             | ..          | ..          | ..                 |


We also have another csv file, which looks like

|     | start_igraph  | end_igraph     | length | lanes   | maxmph | geometry| .. |
|-----|---------------|----------------|--------|---------|--------|---------|----|
| 1   | 1633          | 1631           | 33.77  | 1.0     | 25.0   |LINESTRING (-122.2553076 37.8756414,-122.2554844 37.875675,-122.2555776 37.8756766,-122.2556866 37.8756667)| .. |
| 2   | 1631          | 4552           | 33.77  | 1.0     | 25.0   |LINESTRING (-122.2556866 37.8756667,-122.2555776 37.8756766,-122.2554844 37.875675,-122.2553076 37.8756414)| .. |
| ..  | ..            | ..             | ..     | ..      | ..     |         | .. |

Note that the two tables have two common column names, `start_igraph` and `end_igraph`. Actually, we also know (since we created the CSV files) that records with the same `start_igraph` and `end_igraph` refer to the same road segment in both CSV files. This makes it easier to combine the two tables into one dataframe. Specifically, we will use the `merge()` method to combine the tables. This (as well as many other pandas functions) works similarly to SQL commands if you are familiar with this database management language.

We will learn how to obtain the second csv file during the 7th lecture (September 17).
While you are working on the exercise below, try to answer the following questions:
- What is the shape of the larger_df?
- How many unique street it contains? Note: count unique streets
- What do you see if you go to https://www.openstreetmap.org/node/53055204?
- What do you see if you go to https://www.openstreetmap.org/way/698994084?
- Can you use the above method to quickly locate any road/intersection on OpenStreetMap?

In [5]:
# read the csv file titled `berkeley_osm_edges.csv`, which has more information about the roads in Berkeley
large_df = pd.read_csv('berkeley_osm_edges.csv')

# check some records
display( large_df.loc[ 
            (large_df['start_igraph'].isin([1633, 1631])) & 
            (large_df['end_igraph'].isin([1633, 1631]))] 
       )

,edge_id_igraph,start_igraph,end_igraph,edge_osmid,start_osm,end_osm,length,lanes,maxmph,oneway,type,capacity,crossings_stops,traffic_signals,geometry,start_sp,end_sp,traffic_signals_delay,crossings_stops_delay,fft
4184,4184,1631,1633,698994084,53055203,53055204,33.773753,1.0,25.0,no,residential,950.0,0,0,"LINESTRING (-122.2556866 37.8756667,-122.25557...",1632,1634,0,0.0,3.626393
4188,4188,1633,1631,698994084,53055204,53055203,33.773753,1.0,25.0,no,residential,950.0,0,0,"LINESTRING (-122.2553076 37.8756414,-122.25548...",1634,1632,0,0.0,3.626393


In [6]:
# combine the two dataframes into a new one called `df_expand`
# only keep road segments that are inside `df`
df_expand = df.merge( large_df, how='left', on=['start_igraph', 'end_igraph'] )

print( df_expand.shape )

display( df_expand.head(2) )

(79, 23)


,start_igraph,end_igraph,road_name,direction,road_and_direction,edge_id_igraph,edge_osmid,start_osm,end_osm,length,...,type,capacity,crossings_stops,traffic_signals,geometry,start_sp,end_sp,traffic_signals_delay,crossings_stops_delay,fft
0,1633,1631,Hearst_Ave,Eastward,H_E,4188,698994084,53055204,53055203,33.773753,...,residential,950.0,0,0,"LINESTRING (-122.2553076 37.8756414,-122.25548...",1634,1632,0,0.0,3.626393
1,1631,4552,Hearst_Ave,Eastward,H_E,4185,21468137,53055203,260908845,74.875524,...,residential,950.0,1,0,"LINESTRING (-122.2556866 37.8756667,-122.25577...",1632,4553,0,3.0,8.039619


## Grouping statistics
Another useful operation in Pandas is to aggregate/group records with a common column value and calculate group statistics. In the exertice below, we will practice how to obtain
- The total length of Hearst Avenue and Euclid Avenue (sum up the `length` of segments).
- How many segments are there for each road and directionality?

We will use the `groupby()` method for this exercise. For more approaches to aggredate data, you can refer to https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html.

In [7]:
# first, group the records based on `road_name` using the `groupby()` method
# second, calculate the total `length` of each group using the `agg( )` method on the groupped result
display( df_expand.groupby( 'road_name' ).agg( {'length': 'sum'} ) )

# first, group the records that have the same `road_name` and `directionality` using the `groupby()` method
# second, count the number of records in each group use the `size()` method on the groupped result
# then, use `to_frame()` to give the count column a suitable name
display( df_expand.groupby( ['road_name', 'direction'] ).size().to_frame( 'count' ) )

,length
road_name,
Euclid_Ave,5339.836484
Hearst_Ave,4773.092716


count
road_name  direction       
Euclid_Ave Northward     19
           Southward     19
Hearst_Ave Eastward      21
           Westward      20

## Saving files
The dataframe can be saved to a CSV file, allowing you to work on it using other software (e.g., Excel, Kepler.gl)
- Try dragging the data to https://kepler.gl/demo

In [8]:
df_expand.to_csv('euclid_hearst_expand.csv', index=False)

# Semi-structured data (.json, .xml, .geojson)
JSON (`.json`, `.geojson`) and XML (`.xml`, `.kml`) are also common data file extensions. These types of files have a tree-like hierarchical structure. For example, below is the first few lines of a JSON file containing information of a road network downloaded from the OpenStreetMap's Overpass API (https://overpass-turbo.eu/):
```
{
  "version": 0.6,
  "generator": "Overpass API",
  "elements": [road_1, road_2, ...],
  ...
}
```

In this section, we will learn some basic parsing of such data. Below are the raw road network data for North Berkeley. The extension is `.osm`, but it is still JSON-like. We will learn how to download such data from OpenStreetMaps in the 7th lecture on September 17.

JSON files are read into Python as a dictionary. Try to print out each key (attribute) and value and see what is contained. Try to answer:
- Which attribute contains the largest amount of information? Hint: check the type and length of values for each dictionary key.

In [9]:
# we will handle the `.osm` file using the Python standard library `json`
import json

# read the file with `json.load( open(file_name) )`
osm_data = json.load( open('target_north_berkeley.osm') )

# exploration
print( 'JSON file (target*.osm) is loaded into Python as: ', type( osm_data ) )
print( 'JSON file has the following keys: ', osm_data.keys(), '\n' )
print( "The value of the 'version' key in the dictionary is: ", osm_data['version'] )
print( "The value of the 'element' key in the dictionary has length: ", len(osm_data['elements']) , '\n' )
print( "The first value of the 'element' key in the dictionary is: ", osm_data['elements'][0] )
print( "The last value of the 'element' key in the dictionary is: ", osm_data['elements'][-1] )

JSON file (target*.osm) is loaded into Python as:  <class 'dict'>
JSON file has the following keys:  dict_keys(['version', 'generator', 'osm3s', 'elements']) 

The value of the 'version' key in the dictionary is:  0.6
The value of the 'element' key in the dictionary has length:  6781 

The first value of the 'element' key in the dictionary is:  {'type': 'node', 'id': 35718993, 'lat': 37.8795755, 'lon': -122.2217804}
The last value of the 'element' key in the dictionary is:  {'type': 'way', 'id': 774602461, 'nodes': [7227017062, 4226533420, 53074450], 'tags': {'cycleway': 'shared_lane', 'highway': 'residential', 'lit': 'yes', 'maxspeed': '25 mph', 'name': 'Sonoma Avenue', 'sidewalk': 'both', 'surface': 'asphalt', 'tiger:cfcc': 'A41', 'tiger:county': 'Alameda, CA', 'tiger:name_base': 'Sonoma', 'tiger:name_type': 'Ave'}}


## Extracting and re-organizing the data
After playing around with it, you will possibly realize that the information contained under the 'element' key that we are interested in the most. This part of the data contains the road and intersection information that we need to build a road network. Now let's separate the roads (also known as links/edges/ways) and intersections (also known as nodes). Also, we would only like to keep the following information for roads and intersections:
- Intersection: id, lat and lon
- Road: id and road name

Challenge: can you also extract the start and end coordinates of each road link?
Hint: use dictionary lookup.

Note: intersections are also called nodes. However, not all nodes in the `.osm` data are intersections. Some nodes that are not a real intersection (e.g., where streets intersect) are used to define the geometry position of, e.g., a curvature or a traffic signal.

In [10]:
### create an empty dictionary to hold node information
all_nodes = {}

# this is an example of a road node in the original data
print( osm_data['elements'][0], '\n' )

### loop through the record and add new roads to the `all_nodes` dictionary
for n in osm_data['elements']:
    if n['type']=='node':
        all_nodes[n['id']] = (n['lat'], n['lon'])
print('The data contains {} nodes'.format(len(all_nodes)))

{'type': 'node', 'id': 35718993, 'lat': 37.8795755, 'lon': -122.2217804} 

The data contains 6187 nodes


In [11]:
### create an empty dictionary to hold road link information
all_links = {}

### this is an example of a road link in the original data
print( osm_data['elements'][-1], '\n')

### loop through the record and add new links to the `all_links` dictionary
for l in osm_data['elements']:
    if (l['type']=='way'):
        try:
            all_links[l['id']] = (l['id'], l['tags']['name'])
        except KeyError:
            all_links[l['id']] = (l['id'], '')
print('it includes {} ways'.format(len(all_links)))

{'type': 'way', 'id': 774602461, 'nodes': [7227017062, 4226533420, 53074450], 'tags': {'cycleway': 'shared_lane', 'highway': 'residential', 'lit': 'yes', 'maxspeed': '25 mph', 'name': 'Sonoma Avenue', 'sidewalk': 'both', 'surface': 'asphalt', 'tiger:cfcc': 'A41', 'tiger:county': 'Alameda, CA', 'tiger:name_base': 'Sonoma', 'tiger:name_type': 'Ave'}} 

it includes 594 ways


## Saving the data
After parsing the JSON data, we would like to save our results to output files. These can be done using the `dump()` method. After this, try opening the output files in a text editor and see if the results are what you expect.

In [12]:
with open('nodes.json', 'w') as outfile:
    json.dump(all_nodes, outfile, indent=2)

with open('links.json', 'w') as outfile:
    json.dump(all_links, outfile, indent=2)